In [20]:
import numpy as np
#from autoeisd.utils.parsers import get_backbone_pdb
#from idpconfgen.libs.libcalc import calc_torsion_angles


In [21]:
import os
import time

In [24]:
from Bio.PDB import PDBParser as parser
import pandas as pd

In [25]:
np.random.seed(12)

In [ ]:
def get_backbone_pdb(path, pdb_split=1):
    """
    reads a PDB file with several models and yields backbone structure.

    Parameters
    ----------
    path: str
        path to the pdb file

    pdb_split: int, optional
        length of splits

    Yields
    -------
    ndarray: 3D array of backbone atoms of all pdb data with shape (L, A, 3),
        where L is the `split_length`, and A is the number of atoms.

    """

    # read energy file to dataframe
    parser = PDBParser()
    data = parser.get_structure('data', path)
    models = data.get_models()

    i_count = 0
    n_atoms = 0
    splits = []
    while True:
        try:
            model = next(models)
            i_count += 1


            atoms = []
            for residue in model.get_residues():
                res_name = residue.get_resname()
                atoms += [residue['N'].get_coord(), residue['CA'].get_coord(),
                          residue['C'].get_coord()]

            atoms = np.array(atoms)

            # sanity check
            if i_count == 1:
                n_atoms = atoms.shape[0]
            else:
                assert atoms.shape == (n_atoms, 3)

            splits.append(atoms)

            if i_count % pdb_split == 0:
                splits = np.array(splits)
                assert splits.shape == (pdb_split, n_atoms, 3)
                yield splits
                splits = []

        except StopIteration:
            break


In [ ]:
"""
Copyright: https://github.com/joaomcteixeira/IDPConformerGenerator/blob/cacb4be7effd51da6876ca242ed2250766198453/src/idpconfgen/libs/libcalc.py

"""

def calc_torsion_angles(
        coords,
        ARCTAN2=np.arctan2,
        CROSS=np.cross,
        DIAGONAL=np.diagonal,
        MATMUL=np.matmul,
        NORM=np.linalg.norm,
        ):
    """
    Calculate torsion angles from sequential coordinates.
    Uses ``NumPy`` to compute angles in a vectorized fashion.
    Sign of the torsion angle is also calculated.
    Uses Prof. Azevedo implementation:
    https://azevedolab.net/resources/dihedral_angle.pdf
    Example
    -------
    Given the sequential coords that represent a dummy molecule of
    four atoms:
    >>> xyz = numpy.array([
    >>>     [0.06360, -0.79573, 1.21644],
    >>>     [-0.47370, -0.10913, 0.77737],
    >>>     [-1.75288, -0.51877, 1.33236],
    >>>     [-2.29018, 0.16783, 0.89329],
    >>>     ])
    A1---A2
           \
            \
            A3---A4
    Calculates the torsion angle in A2-A3 that would place A4 in respect
    to the plane (A1, A2, A3).
    Likewise, for a chain of N atoms A1, ..., An, calculates the torsion
    angles in (A2, A3) to (An-2, An-1). (A1, A2) and (An-1, An) do not
    have torsion angles.
    If coords represent a protein backbone consisting of N, CA, and C
    atoms and starting at the N-terminal, the torsion angles are given
    by the following slices to the resulting array:
    - phi (N-CA), [2::3]
    - psi (CA-C), [::3]
    - omega (C-N), [1::3]
    Parameters
    ----------
    coords : numpy.ndarray of shape (N>=4, 3)
        Where `N` is the number of atoms, must be equal or above 4.
    Returns
    -------
    numpy.ndarray of shape (N - 3,)
        The torsion angles in radians.
        If you want to convert those to degrees just apply
        ``np.degrees`` to the returned result.
    """
    # requires
    assert coords.shape[0] > 3
    assert coords.shape[1] == 3

    crds = coords.T
    q_vecs = crds[:, 1:] - crds[:, :-1]
    cross = CROSS(q_vecs[:, :-1], q_vecs[:, 1:], axis=0)
    unitary = cross / NORM(cross, axis=0)

    # components
    u0 = unitary[:, :-1]
    u1 = unitary[:, 1:]
    u3 = q_vecs[:, 1:-1] / NORM(q_vecs[:, 1:-1], axis=0)
    u2 = CROSS(u3, u1, axis=0)
    cos_theta = DIAGONAL(MATMUL(u0.T, u1))
    sin_theta = DIAGONAL(MATMUL(u0.T, u2))

    # torsion angles
    return -ARCTAN2(sin_theta, cos_theta)

In [23]:
asyn_seq = 'MDVFMKGLSKAKEGVVAAAEKTKQGVAEAAGKTKEGVLYVGSKTKEGVVHGVATVAEKTKEQVTNVGGAV\
VTGVTAVAQKTVEGAGSIAAATGFVKKDQLGKNEEGAPQEGILEDMPVDPDNEAYEMPSEEGYQDYEPEA' 
drk_seq = 'MEAIAKHDFSATADDELSFRKTQILKILNMEDDSNWYRAELDGKEGLIPSNYIEMKNHD'

In [217]:
# read JC exp file
exp = pd.read_csv('Desktop/X-EISD/experimental_data/drksh3_JC_exp_clean.txt')
# align torsion index as the first residue doesn't have phi torsion
resn = exp.resnum.values - 2

jc_bc = []
for n in range(200):
    pdb = 'local/pdbs/gen_%i.pdb'%(n+1)
    # calculates JC back calculation alphas from phi torsions
    coords = next(get_backbone_pdb(pdb, 1))[0]
    alpha = np.cos(calc_torsion_angles(coords)[2::3][resn] - np.radians(60))
    jc_bc.append(alpha)


In [236]:
# read NOE exp file
exp = pd.read_csv('Desktop/X-EISD/experimental_data/8AAC_noes.txt')
res1 = exp.res1.values.astype(np.int)
atom1_name = exp.atom1.values
res2 = exp.res2.values.astype(np.int)
atom2_name = exp.atom2.values

multi1 = exp.atom1_multiple_assignments.values
multi2 = exp.atom2_multiple_assignments.values

In [6]:
#read PREs exp file
exp = pd.read_csv('Desktop/X-EISD/experimental_data/drksh3_pres.txt')
res1 = exp.res1.values.astype(np.int)
atom1_name = exp.atom1.values
res2 = exp.res2.values.astype(np.int)
atom2_name = exp.atom2.values

In [228]:
# read EFRET exp file (format modified)
# the EFRET exp file format is based on aSyn repository:
# column names: index, res1, res2, scaler, value
# where scaler is the r0 Foster radius of the dye pair
exp = pd.read_csv('Desktop/X-EISD/experimental_data/drksh3_efret.txt')
res1 = exp.res1.values.astype(np.int)
res2 = exp.res2.values.astype(np.int)
scaler = exp.scale.values

# generates smFRET back calculation data
fret_bc = []
p = parser()
for n in range(200):
    # read pdb files to back calculate
    pdb = 'local/pdbs/gen_%i.pdb'%(n+1)
    struct = p.get_structure('d', pdb)
    # assumes CA as atom labeled
    for j in range(exp.shape[0]):
        r1 = np.int(res1[j])
        r2 = np.int(res2[j])
        d = struct[0]['A'][r1]['CA'] - struct[0]['A'][r2]['CA'] 
        # 58 = abs(res1 - res2) # for drk 
        # scale_factor to adjust for dye size and CA to label distances
        scale_factor = ((np.abs(r1 - r2) + 7) / np.abs(r1 - r2)) ** 0.5
        d = d*scale_factor
        eff = 1.0/(1.0+(d/scaler[j])**6.0)
        #assert isinstance(eff, np.ndarray)
        fret_bc.append(eff)
    if n%100 ==0: time.sleep(5)
fret_bc = np.reshape(fret_bc, (-1, exp.shape[0]))


In [219]:
#PREs
p = parser()
pre_bc = []
for n in range(200):
    pdb = 'local/pdbs/gen_%i.pdb'%(n+1)
    struct = p.get_structure('d', pdb)
    # fetch PRE distances
    dist = []
    for j in range(exp.shape[0]):
        r1 = np.int(res1[j])
        r2 = np.int(res2[j])
        for atom in struct[0]['A'][r1]:            
            if atom1_name[j] == 'H':
                atom1 = struct[0]['A'][r1]['H']
                break
            if atom1_name[j] in atom.get_name():
                atom1 = atom
                break
        for atom in struct[0]['A'][r2]:
            if atom2_name[j] == 'H':
                atom2 = struct[0]['A'][r2]['H']
                break
            if atom2_name[j] in atom.get_name():
                atom2 = atom
                break
        #print(atom1_list, r1, atom1_name[j])

        dist.append(atom1-atom2)
        
    pre_bc.append(dist)
    if n%50 == 0 : time.sleep(5)

In [257]:
# NOEs calculations
p = parser()
noe_bc = []
for n in range(200):
    pdb = 'local/training_25/pdbs/gen_%i.pdb'%(n+1)
    struct = p.get_structure('d', pdb)
    dist = []
    for j in range(exp.shape[0]):
        r1 = np.int(res1[j])
        r2 = np.int(res2[j])
        atom1_list = []
        atom2_list = []
        for atom in struct[0]['A'][r1]:            
            if atom1_name[j] == 'H':
                atom1_list.append(struct[0]['A'][r1]['H'])
                break
            if atom1_name[j] in atom.get_name():
                atom1_list.append(atom)
            if len(atom1_list) == 2: break
            if not multi1[j] and len(atom1_list)==1:
                break
        for atom in struct[0]['A'][r2]:
            if atom2_name[j] == 'H':
                atom2_list.append(struct[0]['A'][r2]['H'])
                break
            if atom2_name[j] in atom.get_name():
                atom2_list.append(atom)
            if len(atom2_list) == 2: break
            if not multi2[j] and len(atom2_list)==1:
                break
        #print(atom1_list, r1, atom1_name[j])
        combos = 0.0
        num_combos = 0
        # handles multiple atom assignments by <r^-6>
        for first_atom in atom1_list:
            for second_atom in atom2_list:
                combos += (first_atom - second_atom)**(-6.)
                num_combos += 1

        dist.append((combos/float(num_combos))**(-1/6))
        
    noe_bc.append(dist)
    if n%50==0: time.sleep(5)

In [3]:
# read chemical shift exp data
exp = pd.read_csv('Desktop/X-EISD/experimental_data/drk_mod_CS.txt')
# chemical shift back calculations using UCBShifts (Li et al. 2020, https://github.com/THGLab/CSpred.git); reading outputs
atommap = {'H':0, 'HA':1, 'C':2, 'CA':3, 'CB':4, 'N':5} #atom order in UCBShifts output
shifts = []
for n in range(450):
    df = pd.read_csv('results/%s.csv'%(n+1))
    for idx in range(exp.shape[0]):
        # fetch chemical shift per datapoint defined in exp
        resnum = int(exp.iloc[idx].resnum)
        atom = atommap[exp.iloc[idx].atomname]+2
        shifts.append(df[df.RESNUM==resnum].values[0, atom])
shifts = np.reshape(shifts, (-1, exp.shape[0]))


In [20]:
# SAXS calculations using crysol; reading outputs
saxs_bc = []
for n in range(20):
    df = pd.read_csv('gen_%s_drk_saxs_exp.fit'%(n+1), delim_whitespace=True)
    saxs_bc.append(df.iloc[:, 3].values)

In [260]:
# save back calculations as csv
#old_alphas = pd.read_csv('Desktop/X-EISD/back_calc_data/noe.txt', header=None, index_col=0)
#new = np.vstack((old_alphas.values, np.array(noe_bc)))
new_alphas = pd.DataFrame(np.array(noe_bc), index=None) 
new_alphas.to_csv('Desktop/X-EISD/back_calc_data/noe.txt', header=None)
